Introduction

Imports & Setup

Data Loading

**Data Pre-processing**

Imports

In [ ]:
import numpy as np
import pandas as pd
import nltk

from nltk.corpus import stopwords
nltk.download('stopwords')

import string

Reading the dataset into a pandas dataframe

In [21]:
df = pd.read_csv('spam.csv', encoding='latin-1')

print(df.shape)
df.head()

(5572, 5)


,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


Renaming columns and removing uneccessary ones

In [22]:
# Renaming first two columns and removing others
df = df[['v1', 'v2']]
df.columns = ['Label', 'SMS']

df.head()

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


Feature Extraction

Model Training

Evaluation

Testing on Custom Messages